In [ ]:
vlib = c("tidyverse", "ggpubr", "arrow", "ComplexHeatmap", "Seurat",
         "tidyseurat", "enrichR", "SeuratData", "fgsea", "msigdbr",
         "presto", "patchwork", "ggrepel", "hdWGCNA")
lapply(vlib, require, character.only = TRUE, quietly = TRUE) |> suppressMessages()

base_dir = "."
setwd(base_dir)

saveRDS.gz <- function(object, file, threads = 4) {
  con <- pipe(paste0("pigz -p", threads, " > ", file), "wb")
  saveRDS(object, file = con)
  close(con)
}
readRDS.gz <- function(file, threads = parallel::detectCores()) {
  con <- pipe(paste0("pigz -d -c -p", threads, " ", file))
  object <- readRDS(file = con)
  close(con)
  return(object)
}

## figure 4A

In [ ]:
df_module = list.files("scRNAseq_downstream_scenic/DME_baseline/", ".txt.gz", full.names = TRUE)
df_module = df_module[!grepl("pseudobulk", df_module)] %>% lapply(read_delim)
names(df_module) = c("B", "CD4", "CD8", "DC", "Mono", "NK", "Other_T", "Other")
c_colors = c(
  "B" = "#C0392BFF",
  "CD4" = "#D35400FF", 
  "CD8" = "#F39C12FF",
  "DC" = "#27AE60FF",
  "Mono" = "#16A085FF",
  "NK" = "#2980B9FF",
  "Other" = "#8E44ADFF",
  "Other_T" = "#2C3E50FF"
)

p_DME = df_module %>% bind_rows(.id = "cluster_name") %>%
  filter(cluster_name %in% c("B", "CD4", "CD8", "NK", "Mono", "Other_T")) %>%
  arrange(padj) %>%
  mutate(label = ifelse(padj < 1e-50,
                        paste0(cluster_name, " ", feature),
                        NA)) %>%
  filter(group == "PD") %>%
  mutate(feature = ifelse(padj < 1e-5, feature, NA)) %>%
  ggplot(aes(y = -log10(padj), x = logFC)) +
  geom_point(aes(color = cluster_name), size = 2.5, shape = 16) +
  geom_text_repel(aes(label = label)) +
  xlim(c(-2, 2)) +
  xlab("estimate on PD") +
  ylab("-log10(adjusted pval)") +
  theme_pubr(legend = 'right') +
  scale_color_manual(values = c_colors)

p_DME %>%
  ggsave("./figure_prep/p_DME_volcano.pdf", ., width = 4, height = 3, dpi = 300)

## figure 3B

In [ ]:
# gsea analysis
library(fgsea)
library(msigdbr)
m_df = msigdbr(species = "Homo sapiens", category = "H") 
fgsea_sets =  m_df %>% split(x = .$gene_symbol, f = .$gs_name)

df_gsea_res = read_delim("assets/wgcna/module_gene_scored.CD8.RNA.txt.gz")
  filter(color == "brown") %>%
  select(gene_name, value) %>%
  deframe() %>%
  fgsea(fgsea_sets, ., scoreType = "pos") 

df_gsea_res_forPlot = df_gsea_res %>%
  arrange(padj) %>%
  top_n(n = 10, -pval) %>%
  mutate(pathway = str_to_sentence(str_sub(pathway, 10, -1))) %>%
  mutate(pathway = str_replace_all(pathway, "_", " "),
         name = c_cluster) %>%
  mutate(pathway = factor(pathway, levels = rev(unique(.$pathway))),
         color_size = ifelse(padj > 0.05, NA, size))

pGSEA_each = df_gsea_res_forPlot %>%
  mutate(name = paste0("CD8_", c_module)) %>%
  ggplot(aes(x =  NES, size = -log10(padj), y = pathway, color = color_size)) +
  geom_point() + scale_color_gradient2(low ="#BCE4D8",
                                       mid = "#3E9AB4",
                                       high = "#2C5985",
                                       midpoint = 20,
                                       na.value = "grey70") +
  theme_pubr(legend = "right") +
  facet_wrap(~name) +
  theme(strip.background = element_rect(fill= NA, linewidth = 1),
        strip.text = element_text(size = 13)) +
  scale_x_continuous(limits = c(1, 2.5), breaks = c(1, 2.0))

fname_out = paste0("figure_prep/p_DME_CD8_", c_module, "_gsea.pdf")
options(repr.plot.width = 7, repr.plot.height = 5, repr.plot.res = 150)
pGSEA_each
pGSEA_each %>% ggsave(fname_out, ., width = 18, height = 8, units = "cm", dpi = 300)

In [ ]:
pModuleExprs_byCondition = read_delim("assets/wgcna/CD8_MEs.txt.gz", delim = "\t") %>%
  mutate(time = ifelse(time == "base", "baseline", "treatment")) %>%
  mutate(group = factor(paste(Response, time), levels = c("PR baseline", "PD baseline", "PR treatment", "PD treatment")),
         grid_name = "CD8 brown") %>%
  ggplot(aes(x = group, y = get(c_module))) +
  xlab("group") + ylab('module expression') +
  theme_pubr(x.text.angle = 45) +
  geom_violin(aes(fill = group)) +
  geom_boxplot(width = 0.50, alpha = 0.5, outliers = FALSE) +
  facet_wrap(~grid_name) +
  scale_fill_manual(values = c("grey90", "#E0958C", "grey70", "#C65E5A")) +
  theme(strip.background = element_rect(fill= NA, linewidth = 1),
        strip.text = element_text(size = 13))
options(repr.plot.width = 4, repr.plot.height = 5, repr.plot.res = 150)
pModuleExprs_byCondition
pModuleExprs_byCondition %>%
  ggsave("figure_prep/p_DME_byCondition.pdf", ., width = 7,
         height = 10, dpi = 300, units = "cm")